# ♻️ EcoScan Hybrid SSL + RL Notebook
This notebook integrates Self-Supervised Learning (SSL) and Reinforcement Learning (RL) to pseudo-label a dataset and optimize reward point assignment using confidence-aware policy training.

In [ ]:
# STEP 1: Install Required Libraries
!pip install datasets transformers torch torchvision pillow stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade gcsfs fsspec==2025.3.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.1 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [ ]:
from huggingface_hub import login
login()  # Paste your token when prompted


In [ ]:
# STEP 2: Load Pretrained Model and Dataset
from transformers import AutoModelForImageClassification, AutoImageProcessor
from datasets import load_dataset
from PIL import Image
import torch
from tqdm import tqdm

model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Recycling-Net-11")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Recycling-Net-11")
labels = model.config.id2label
model.eval()

dataset = load_dataset("viola77data/recycling-dataset")['train']

Resolving data files:   0%|          | 0/3107 [00:00<?, ?it/s]

102_3404fae2.jpg:   0%|          | 0.00/18.9k [00:00<?, ?B/s]

114_0acfffba.jpg:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

118_e47ebf39.jpg:   0%|          | 0.00/50.0k [00:00<?, ?B/s]

108_a0c11114.jpg:   0%|          | 0.00/18.9k [00:00<?, ?B/s]

...

299_4011dd52.jpg:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

298_cf9555cb.jpg:   0%|          | 0.00/29.7k [00:00<?, ?B/s]

300_7a6f57b5.jpg:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

297_fd0a4e76.jpg:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3107 [00:00<?, ? examples/s]

In [ ]:
# STEP 3: Pseudo-label Dataset with Confidence Threshold
pseudo_labeled_data = []
threshold = 0.75

def predict_with_confidence(image):
    image = image.convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        confidence, pred = torch.max(probs, dim=1)
        return pred.item(), confidence.item()

for example in tqdm(dataset):
    image = example['image']
    label, confidence = predict_with_confidence(image)
    if confidence > threshold:
        pseudo_labeled_data.append((image, label, confidence))
print(f"Collected {len(pseudo_labeled_data)} pseudo-labeled images.")

100%|██████████| 3107/3107 [44:34<00:00,  1.16it/s]

Collected 2466 pseudo-labeled images.


In [ ]:
# STEP 4: Define RL Environment for Reward Assignment
from gym import Env, spaces
import numpy as np
import random

class RewardEnv(Env):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.index = 0
        self.action_space = spaces.Discrete(21)  # Points: 5 to 25
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)

    def reset(self):
        self.index = random.randint(0, len(self.data)-1)
        return [self.data[self.index][2]]  # confidence

    def step(self, action):
        points = 5 + action
        label = self.data[self.index][1]
        reward = 1 if points >= 10 and label != 4 else -1
        return [self.data[self.index][2]], reward, True, {}

In [ ]:
!pip install "shimmy>=2.0"


In [ ]:
# STEP 5: Train PPO Agent
from stable_baselines3 import PPO

env = RewardEnv(pseudo_labeled_data)
model_rl = PPO('MlpPolicy', env, verbose=1)
model_rl.learn(total_timesteps=3000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.34     |
| time/              |          |
|    fps             | 382      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | 0.56       |
| time/                   |            |
|    fps                  | 259        |
|    iterations           | 2          |
|    time_elapsed         | 15         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.10811658 |
|    clip_fraction        | 0.571      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.99      |
|    explained_variance   | -0.000243  |
|    learning_rate        | 0.0003     |
|   

In [ ]:
# STEP 6: Use RL Agent to Assign Smart Rewards
for image, label, conf in pseudo_labeled_data[:10]:
    obs = [conf]
    action, _ = model_rl.predict(obs)
    points = 5 + action
    print(f"Label: {labels[label]} | Confidence: {conf:.2f} | Assigned Points: {points}")

Label: aluminium | Confidence: 0.81 | Assigned Points: 13
Label: aluminium | Confidence: 1.00 | Assigned Points: 14
Label: aluminium | Confidence: 0.99 | Assigned Points: 20
Label: aluminium | Confidence: 0.94 | Assigned Points: 21
Label: aluminium | Confidence: 1.00 | Assigned Points: 13
Label: aluminium | Confidence: 0.76 | Assigned Points: 25
Label: aluminium | Confidence: 0.96 | Assigned Points: 18
Label: aluminium | Confidence: 0.99 | Assigned Points: 24
Label: aluminium | Confidence: 0.99 | Assigned Points: 18
Label: aluminium | Confidence: 0.98 | Assigned Points: 10


In [ ]:
model.save_pretrained("model/")
processor.save_pretrained("model/")


['model/preprocessor_config.json']

In [ ]:
import shutil
shutil.make_archive("EcoScan_Model", 'zip', "model/")


'/content/EcoScan_Model.zip'

In [ ]:
from google.colab import files
files.download("EcoScan_Model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>